In [ ]:
# !pip install pandas
# !pip install pinecone
# !pip unstall openai
# !pip install python-dotenv
# !pip install scikit-learn

In [3]:
from openai import OpenAI
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
def get_chatbot_response(client,model_name,messages,temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000,
    ).choices[0].message.content
    
    return response

In [5]:
client = OpenAI(
            api_key=os.getenv("RUNPOD_TOKEN"),
            base_url=os.getenv("RUNPOD_CHATBOT_URL"),
        )
model_name = os.getenv("MODEL_NAME")

# Get LLM response

In [6]:
messages = [{'role':'user','content':"What's the capital of Italy?"}]
response = get_chatbot_response(client,model_name,messages)
print(response)

The capital of Italy is Rome.


# Prompt engineering

## Structred output

In [7]:
system_prompt = """
You are a helpful AI assistant for a coffee shop application which serves drinks and pastries.
            Your task is to determine whether the user is asking something relevant to the coffee shop or not.
            The user is allowed to:
            1. Ask questions about the coffee shop, like location, working hours, menue items and coffee shop related questions.
            2. Ask questions about menue items, they can ask for ingredients in an item and more details about the item.
            3. Make an order.
            4. ASk about recommendations of what to buy.

            The user is NOT allowed to:
            1. Ask questions about anything else other than our coffee shop.
            2. Ask questions about the staff or how to make a certain menue item.

            Your output should be in a structured json format like so. each key is a string and each value is a string. Make sure to follow the format exactly:
            {
            "chain of thought": go over each of the points above and make see if the message lies under this point or not. Then you write some your thoughts about what point is this input relevant to.
            "decision": "allowed" or "not allowed". Pick one of those. and only write the word.
            "message": leave the message empty if it's allowed, otherwise write "Sorry, I can't help with that. Can I help you with your order?"
            }
"""
input_messages = [{"role": "system", "content": system_prompt}] + messages[-3:]
messages.append({'role':'user','content':"I want to order a latte"})
response = get_chatbot_response(client,model_name,messages)
print(response)

In Italy, when ordering a latte, you might be surprised to know that it's actually a type of milk, not a coffee drink. If you want a coffee drink, you would typically order a "caffè latte" or simply "latte macchiato" if you want a shot of espresso "marked" with a small amount of milk.

However, if you're in the US or other countries, a "latte" usually refers to a coffee drink made with espresso and steamed milk. In that case, you can simply order a "latte" and the barista will know what you mean.

To clarify, you can say:

* "Un caffè latte, per favore" (a coffee latte, please) in Italy
* "A latte, please" in the US or other countries


In [11]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'}]

## input structuring

In [12]:
user_prompt = """
Get me the capitals of the following countries:
```
1. Italy
2. France
3. Germany
``
"""
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':user_prompt})
response = get_chatbot_response(client,model_name,messages)
print(response)

[
  {
    "country": "Italy",
    "capital": "Rome"
  },
  {
    "country": "France",
    "capital": "Paris"
  },
  {
    "country": "Germany",
    "capital": "Berlin"
  }
]


In [13]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'},
 {'country': 'France', 'capital': 'Paris'},
 {'country': 'Germany', 'capital': 'Berlin'}]

## Give the model time to think (Chain of thought)

> https://arxiv.org/pdf/2205.11916

In [18]:
user_prompt = """
Calculate the result of this equation: 1+3

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "result": 4
}


In [17]:
259/2*8654+91072*33-12971

4113098.0

In [15]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "result": 1431449.0
}


In [19]:
4113098.0 - 1431449.0

2681649.0

In [29]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to final result. Feel free to write here in free text. 
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)

In [30]:
print(response)

{
    "steps": "First, we need to follow the BEDMAS order of operations. 
    1. Divide 259 by 2: 259/2 = 129.5
    2. Multiply 129.5 by 8654: 129.5 * 8654 = 1121011
    3. Multiply 91072 by 33: 91072 * 33 = 3005016
    4. Add 1121011 and 3005016: 1121011 + 3005016 = 4126027
    5. Subtract 12971 from 4126027: 4126027 - 12971 = 4113056",
    "result": 4113056
}


In [33]:
4113098.0 - 4113056

42.0

# RAG - Retrieval Augmented Generation

#### Asking about a subject that the LLM does not know anything about

In [5]:
user_prompt = """
What's new in iphone 16?
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

Since I'm not aware of any official information about the iPhone 16, I'll provide you with some general information about the latest iPhone models and some rumored features that might be included in future iPhone models.

However, I can tell you about the latest iPhone models, such as the iPhone 14 series, which includes:

1. iPhone 14
2. iPhone 14 Plus
3. iPhone 14 Pro
4. iPhone 14 Pro Max

Some of the key features of the iPhone 14 series include:

1. Improved cameras with a new 48MP main camera on the Pro models
2. Faster A16 Bionic chip
3. Longer battery life
4. New colors and designs
5. Enhanced display with a higher refresh rate

As for the iPhone 16, there's no official information available yet. However, based on rumors and leaks, here are some potential features that might be included:

1. Improved cameras with a new periscope lens or a 3D modeling camera
2. Faster A17 Bionic chip or a new chip design
3. Enhanced display with a higher refresh rate or a new OLED panel
4. New col

#### Giving Context to the unknown subject

In [6]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [7]:
user_prompt = f"""
{iphone_16}

What's new in iphone 16?
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

According to the text, the new features and updates in the iPhone 16 include:

1. Larger display sizes:
   - Base model: 6.1-inch
   - iPhone 16 Plus: 6.7-inch
   - iPhone 16 Pro: 6.3-inch
   - iPhone 16 Pro Max: 6.9-inch

2. Thinner bezels and a more durable Ceramic Shield.

3. New A18 chip (A18 Pro for Pro models) for improved performance, with:
   - Enhanced neural engine capabilities
   - Faster GPU for gaming
   - Machine learning tasks

4. Upgraded camera systems:
   - Base iPhone 16: Dual-camera setup with a 48MP main sensor
   - Pro models: 48MP Ultra Wide and 5x telephoto camera, with Apple's "Camera Control" button

5. Advanced audio features:
   - "Audio Mix" for refined audio capture during video recording

6. Extended battery life, especially in the iPhone 16 Pro Max.

7. Switch to USB-C for faster charging and data transfer.

8. Support for up to 2x faster video encoding in Pro models.

9. Starting prices remain consistent with previous generations:
   - iPhone 16: $799
 

#### Automatically extract context data from database

In [8]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [9]:
data = [iphone_16,samsung_s23]

In [10]:
user_prompt = """What's new in iphone 16?"""

In [6]:
embedding_client = OpenAI(
        api_key=os.getenv("RUNPOD_TOKEN"), 
        base_url=os.getenv("RUNPOD_EMBEDDING_URL")
    )

In [7]:
def get_embedding(embedding_client,model_name,text_input):
    output = embedding_client.embeddings.create(input = text_input,model=model_name)
    
    embedings = []
    for embedding_object in output.data:
        embedings.append(embedding_object.embedding)

    return embedings

In [8]:
user_prompt_embeddings = get_embedding(embedding_client,model_name,user_prompt)

In [9]:
user_prompt_embeddings

[[-0.05107622966170311,
  -0.03489547595381737,
  0.06362394988536835,
  -0.008907047100365162,
  -0.03724626451730728,
  -0.0420089028775692,
  0.015967046841979027,
  0.031155584380030632,
  -0.012799587100744247,
  0.04243631660938263,
  0.05556409806013107,
  0.013288062997162342,
  -0.07217226922512054,
  0.00139673484954983,
  0.09635180979967117,
  0.028697941452264786,
  0.11521918326616287,
  -0.14751963317394257,
  -0.07907198369503021,
  0.016165489330887794,
  -0.03666619956493378,
  -0.014616105705499649,
  -0.039841290563344955,
  -0.030025985091924667,
  0.030025985091924667,
  0.061517395079135895,
  0.0010771268280223012,
  -0.012753792107105255,
  -0.019157402217388153,
  -0.13848282396793365,
  -0.000405234401114285,
  -0.006754701491445303,
  0.01666923053562641,
  0.026835627853870392,
  0.00040475736022926867,
  -0.038497984409332275,
  -0.0031502859201282263,
  0.03492600470781326,
  -0.009632128290832043,
  0.06380712240934372,
  0.029629098251461983,
  0.091039

In [15]:
user_prompt_embeddings = user_prompt_embeddings[0]

In [16]:
data_embeddings = [get_embedding(embedding_client,model_name,x)[0] for x in data]

In [17]:
data_similaraty_scores = cosine_similarity([user_prompt_embeddings], data_embeddings)

In [20]:
data_similaraty_scores

array([[0.73779758, 0.52622817]])

In [22]:
closest_entry_index=data_similaraty_scores.argmax()
closest_entry_index

0

In [23]:
data[closest_entry_index]

'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine l

In [24]:
user_prompt_with_data = f"""
{data[closest_entry_index]}

{user_prompt}
""" 

In [25]:
user_prompt_with_data

'\n\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine

In [26]:
messages = [{'role':'user','content':user_prompt_with_data}]
response = get_chatbot_response(client,model_name,messages)
print(response)

According to the text, the new features and updates in the iPhone 16 include:

1. Larger display sizes:
   - Base model: 6.1-inch
   - iPhone 16 Plus: 6.7-inch
   - iPhone 16 Pro: 6.3-inch
   - iPhone 16 Pro Max: 6.9-inch

2. Thinner bezels and a more durable Ceramic Shield.

3. New A18 chip (A18 Pro for Pro models) for improved performance, with:
   - Enhanced neural engine capabilities
   - Faster GPU for gaming
   - Machine learning tasks

4. Upgraded camera systems:
   - Base iPhone 16: Dual-camera setup with a 48MP main sensor
   - Pro models: 48MP Ultra Wide and 5x telephoto camera, with Apple's "Camera Control" button

5. Advanced audio features:
   - "Audio Mix" for refined audio capture during video recording

6. Extended battery life, especially in the iPhone 16 Pro Max.

7. Switch to USB-C for faster charging and data transfer.

8. Support for up to 2x faster video encoding in Pro models.

9. Starting prices remain consistent with previous generations:
   - iPhone 16: $799
 